In [1]:
! pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.8 MB/s eta 0:00:00


In [2]:
from langgraph.graph import StateGraph,START,END

In [3]:
import os
from getpass import getpass

os.environ["HUGGINGFACEHUB_ACCESS_TOKEN"] = getpass("Enter your secret: ")

Enter your secret: ··········


In [5]:
!pip install langchain langchain-community langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [6]:
# Define the model
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1",
    task="text-generation",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_ACCESS_TOKEN"],
)

model = ChatHuggingFace(llm=llm)

In [7]:
from typing import TypedDict

class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [8]:
def create_outline(state:BlogState)->BlogState:
    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [9]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [10]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [11]:
intial_state = {'title': 'Rise of AI in India'}
final_state = workflow.invoke(intial_state)

print(final_state)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'title': 'Rise of AI in India', 'outline': "<think>\nOkay, so the user wants me to generate a detailed outline for a blog about the rise of AI in India. Hmm, first, I need to make sure I cover all the important aspects. Let's start with an introduction to set the context. India has been making strides in technology, so explaining how AI is part of that growth makes sense.\n\nI should break down the outline into sections. Maybe start with an intro that defines AI and gives an overview of its growth in India. Then, move into the historical context. When did AI start gaining traction there? Important milestones would be good here, like government initiatives or key projects.\n\nNext, I should consider the industries adopting AI. Healthcare, agriculture, finance, education, retail, and manufacturing come to mind. Each sector can be a subpoint with examples of Indian companies or programs using AI. For instance, in healthcare, there are startups using AI for diagnostics. In agriculture, ma

In [12]:
print(final_state['outline'])

<think>
Okay, so the user wants me to generate a detailed outline for a blog about the rise of AI in India. Hmm, first, I need to make sure I cover all the important aspects. Let's start with an introduction to set the context. India has been making strides in technology, so explaining how AI is part of that growth makes sense.

I should break down the outline into sections. Maybe start with an intro that defines AI and gives an overview of its growth in India. Then, move into the historical context. When did AI start gaining traction there? Important milestones would be good here, like government initiatives or key projects.

Next, I should consider the industries adopting AI. Healthcare, agriculture, finance, education, retail, and manufacturing come to mind. Each sector can be a subpoint with examples of Indian companies or programs using AI. For instance, in healthcare, there are startups using AI for diagnostics. In agriculture, maybe crop prediction tools.

The AI ecosystem in In